# Single model for classification and regression

![image-6](image-6.png)


In [3]:
from tensorflow.keras.layers import Input, Dense

# Input layer
input_tensor = Input(shape=(1,))

# Output regression layer
output_tensor_reg = Dense(1)(input_tensor)

# Output classification layer
output_tensor_class = Dense(1, activation='sigmoid')(output_tensor_reg)

In [5]:
from tensorflow.keras.models import Model

# Instantiate the model
model = Model(input_tensor, [output_tensor_reg, output_tensor_class])

# With two output models, each output needs its own loss function
model.compile(loss=['mean_absolute_error','binary_crossentropy'],
             optimizer='adam')

# Model summary
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 2         
                                                                 
 dense_4 (Dense)             (None, 1)                 2         
                                                                 
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


## Fit the combination classifier/regerssor

In [6]:
import pandas as pd

# Read tournament data
games_tourney = pd.read_csv('datasets/games_tourney.csv')
games_tourney

,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won
0,1985,288,73,0,-3,-9,41,50,0
1,1985,5929,73,0,4,6,61,55,1
2,1985,9884,73,0,5,-4,59,63,0
3,1985,73,288,0,3,9,50,41,1
4,1985,3920,410,0,1,-9,54,63,0
...,...,...,...,...,...,...,...,...,...
4229,2017,10120,10696,0,-1,-10,74,84,0
4230,2017,320,10818,0,9,-2,71,73,0
4231,2017,2910,10818,0,8,-25,66,91,0
4232,2017,3300,10818,0,10,24,83,59,1


In [7]:
# Features and targets
X = games_tourney['seed_diff']

y = games_tourney[['score_diff','won']]

# Split train-test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X,y,
                                                 test_size=0.3,
                                                 random_state=42)

In [9]:
# Fit the model
model.fit(X_train, [y_train['score_diff'], y_train['won']],
         epochs=20,
         batch_size=64)

Epoch 1/20
47/47 [==============================] - 0s 918us/step - loss: 20.8953 - dense_3_loss: 16.2111 - dense_4_loss: 4.6842
Epoch 2/20
47/47 [==============================] - 0s 907us/step - loss: 20.2324 - dense_3_loss: 15.9757 - dense_4_loss: 4.2568
Epoch 3/20
47/47 [==============================] - 0s 899us/step - loss: 19.6069 - dense_3_loss: 15.7444 - dense_4_loss: 3.8624
Epoch 4/20
47/47 [==============================] - 0s 947us/step - loss: 19.0137 - dense_3_loss: 15.5156 - dense_4_loss: 3.4981
Epoch 5/20
47/47 [==============================] - 0s 962us/step - loss: 18.4498 - dense_3_loss: 15.2902 - dense_4_loss: 3.1596
Epoch 6/20
47/47 [==============================] - 0s 936us/step - loss: 17.9215 - dense_3_loss: 15.0705 - dense_4_loss: 2.8509
Epoch 7/20
47/47 [==============================] - 0s 2ms/step - loss: 17.4283 - dense_3_loss: 14.8577 - dense_4_loss: 2.5707
Epoch 8/20
47/47 [==============================] - 0s 2ms/step - loss: 16.9661 - dense_3_loss: 14.

In [10]:
# Look at the model's weights
model.get_weights()

[array([[-0.20029022]], dtype=float32),
 array([-0.0086361], dtype=float32),
 array([[0.3244505]], dtype=float32),
 array([-0.00801555], dtype=float32)]

In [11]:
# Evaluate the model
model.evaluate(X_test, [y_test['score_diff'],y_test['won']])

40/40 [==============================] - 0s 776us/step - loss: 13.2741 - dense_3_loss: 12.4384 - dense_4_loss: 0.8357


[13.27409839630127, 12.438377380371094, 0.8357194662094116]